In [7]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mne

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
df = pd.read_csv("RecordedSessions/Antony-2024-03-16-11-37-59.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
# df = pd.read_csv("UnicornRecorder_20240104_143245.csv")
# df.describe()

In [ ]:
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
from scipy.signal import butter, sosfiltfilt, iirnotch, lfilter

def preprocess_eeg(df, sfreq, lowcut=0.5, highcut=30, notch_freq=50):
    # Convert DataFrame to NumPy array if necessary
    if isinstance(df, pd.DataFrame):
        data = df.values
    else:
        data = df  # Assuming data is already a NumPy array

    # Apply Bandpass Filter
    sos = butter(N=5, Wn=[lowcut, highcut], btype='bandpass', fs=sfreq, output='sos')
    filtered = sosfiltfilt(sos, data, axis=-1)
    
    # Apply Notch Filter
    b, a = iirnotch(w0=notch_freq/(sfreq/2), Q=30)
    filtered_notched = lfilter(b, a, filtered, axis=-1)
    
    # ICA for artifact removal (Example Steps)
    # 1. Create MNE Raw object
    ch_names = ['EEG' + str(i+1) for i in range(filtered_notched.shape[0])]
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
    raw = mne.io.RawArray(filtered_notched, info)

    # 2. Fit ICA
    ica = ICA(n_components=min(8, data.shape[0]), random_state=97)  # Adjusted n_components
    ica.fit(raw)

    # Note: Manual component selection for artifact rejection is recommended here.
    # ica.plot_components()
    # ica.exclude = [bad_components]  # Specify bad components to exclude
    # ica.apply(raw)

    return raw.get_data()  # This returns the processed data as a NumPy array

# Usage example:
# If your data is in a DataFrame `df` and `sfreq` is the sampling frequency
# processed_data = preprocess_eeg(df, sfreq=256)

df = preprocess_eeg(df.iloc[:,:8].transpose(), 250)


In [ ]:
df.shape

In [ ]:
plt.plot(df[0][300:])

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import butter, sosfilt, sosfreqz, iirnotch, lfilter

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band', output='sos')
    return sos

def apply_notch_filter(data, fs, freq, quality=30):
    nyq = 0.5 * fs
    w0 = freq / nyq
    b, a = iirnotch(w0, quality)
    y = lfilter(b, a, data)
    return y

# Your DataFrame (df) goes here
# Assuming df is your DataFrame and fs is your sampling rate
fs = 250  # Example sampling rate, replace with your actual sampling rate
lowcut = 0.5
highcut = 30

# Applying the bandpass filter to the first 8 columns
sos = butter_bandpass(lowcut, highcut, fs)
df.iloc[:, :8] = df.iloc[:, :8].apply(lambda x: sosfilt(sos, x), axis=0)

# Applying the notch filter at 50Hz to the already bandpass filtered data
df.iloc[:, :8] = df.iloc[:, :8].apply(lambda x: apply_notch_filter(x, fs, 50), axis=0)

# Now df contains the first 8 columns filtered according to your specifications


In [ ]:
df[df.columns[1]]

In [ ]:
df.describe()

In [ ]:
for i in range(8):
    fig, ax = plt.subplots(figsize=(20,8))
    ax.plot(df[df.columns[i]].iloc[:1000])

In [ ]:
means = []

for i in range(1250):
    vals = []
    for j in range(8):
        vals.extend(df[df.Counter == i][df.columns[j]].values)
    # print(vals)
    means.append(np.mean(np.array(vals)))
    

In [ ]:
import matplotlib.pyplot as plt

n = means[:]

# Create an index for each value (assuming you want to plot them in the order they appear)
indexes = range(len(n))

# Plotting the bar graph
plt.figure(figsize=(20, 8))  # Optional: Adjust figure size
plt.bar(indexes, n)

# Optionally, you can customize the ticks on the x-axis to show meaningful labels or simply the indexes
plt.xticks(indexes, [f'Item {i+1}' for i in indexes])

plt.xlabel('Item')  # Customize with your actual label
plt.ylabel('Value')  # Customize with your actual label
plt.title('Bar Graph of Values')  # Customize with your actual title
plt.show()


In [ ]:
df.info()

In [ ]:
df_excludeStart = df[df[" State"] == "Right"]#df[df[" State"] == "Up"]#[df['Counter'] >= 20]
df_excludeStart

In [ ]:
df_excludeStart.describe()

In [ ]:
for i in range(8):
    fig, ax = plt.subplots(figsize=(20,8))
    ax.plot(df_excludeStart[df_excludeStart.columns[i]].iloc[:4000])

In [ ]:
# Open the file in binary mode
with open('data (5).bin', 'rb') as f:
    # Read the data into a NumPy array
    array = np.fromfile(f, dtype=np.float32)  # Change dtype according to your data

In [ ]:
array.shape

In [ ]:
array = array.reshape(-1,17)
array.shape

In [ ]:
array = pd.DataFrame(array)

In [ ]:
array

In [ ]:
array.describe()

In [ ]:
plt.plot(array[array.columns[8]])